### Fake News Classifier Using LSTM

In [1]:
# Loadig Libraries

import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

import nltk
import re
from nltk.corpus import stopwords

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Data_Science_Work_Space/NLP/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [3]:
df = df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
df.shape

(18285, 5)

In [ ]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
# get the independent features

X = df.drop(['label'],axis=1)

# get the dependent fetures

y= df['label']

In [10]:
# vocabulary Size
voc_size=5000

In [11]:
# One hot representation

message = X.copy()

In [ ]:
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [12]:
message.reset_index(inplace=True)

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [36]:
ps = PorterStemmer()
lm = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

corpus=[]
for i in range(0,len(message)):
  review = re.sub('[^a-zA-Z]', ' ',message['title'][i])
  review = review.lower()
  review = review.split()

  # review = [ps.stem(word) for word in review if not word in stopwords.words('english') ] # stemming
  review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english') ] # lemma
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [37]:
corpus[0:5]

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery']

In [38]:
onehot_rep = [one_hot(word,voc_size) for word in corpus]
onehot_rep[0]

[3940, 4667, 134, 4498, 3032, 1479, 504, 429, 4670, 4803]

In [39]:
# Padding
sent_length= 20
embedded_docs = pad_sequences(onehot_rep,padding='post',maxlen=sent_length)
embedded_docs

array([[3940, 4667,  134, ...,    0,    0,    0],
       [4129, 4507, 2202, ...,    0,    0,    0],
       [3197, 3228, 3807, ...,    0,    0,    0],
       ...,
       [3448, 2843, 4084, ...,    0,    0,    0],
       [ 284, 2504, 4251, ...,    0,    0,    0],
       [1928, 4319, 2223, ...,    0,    0,    0]], dtype=int32)

In [40]:
embedded_docs[0]

array([3940, 4667,  134, 4498, 3032, 1479,  504,  429, 4670, 4803,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [41]:
# creating model

embedding_vector_features=100
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 100)           500000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embedded_docs,y,test_size=0.33,random_state=42)

In [44]:
# final training

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0092 - accuracy: 0.9967 - val_loss: 0.6008 - val_accuracy: 0.8920
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0105 - accuracy: 0.9964 - val_loss: 0.5203 - val_accuracy: 0.9011
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.4683 - val_accuracy: 0.8994
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 0.6450 - val_accuracy: 0.8983
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.6183 - val_accuracy: 0.9057
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0069 - accuracy: 0.9979 - val_loss: 0.6031 - val_accuracy: 0.9016
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0056 - accuracy: 0.9982 - val_loss: 0.5591 - val_accuracy: 0.9006
Epoch 

In [49]:
# Accuracy check
import numpy as np

y_pred = model.predict(X_test)

y_pred= np.where(y_pred>0.2,1,0) # AUC ROC curve  

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9005799502899752